**LongContextReorder**

모델의 아키텍처와 상관없이, 10개 이상의 검색된 문서를 포함할 경우 성능이

상당히 저하됨.

간단히 말해, 모델이 긴 컨텍스트 중간에 있는 관련 정보에 접근해야 할 때,

제공된 문서를 무시하는 경향이 있음.

이 문제를 피하기 위해, 검색 후 문서의 순서를 재배열하여 성능 저하를

방지할 수 있음.

In [1]:
# ============================
# 1) 설치
# ============================
!pip -q install langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [1]:

# ============================
# 2) PDF 다운로드 (재시도 포함)
# ============================
import requests, os, time
import urllib.request


In [ ]:

urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")


In [6]:

pdf_path = "2020_경제금융용어 700선_게시.pdf"

max_retries = 3
for attempt in range(max_retries):
    if not os.path.exists(pdf_path) or os.path.getsize(pdf_path) == 0:
        try:
            print(f" Downloading PDF (Attempt {attempt+1}/{max_retries})")
            r = requests.get(url, stream=True, timeout=30)
            r.raise_for_status()
            with open(pdf_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            if os.path.exists(pdf_path) and os.path.getsize(pdf_path) > 0:
                print(" PDF downloaded:", pdf_path)
                break
        except requests.exceptions.RequestException as e:
            print(" Download failed:", e)
            time.sleep(2)
    else:
        print(" PDF already exists:", pdf_path)
        break
else:
    raise RuntimeError(" Failed to download PDF after retries.")

# ============================
# 3) PDF 로드 & 청크 분할
# ============================
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f" 전체 페이지 수: {len(docs)}")
print(f" 청크 수: {len(chunks)}")
print("샘플 청크:", chunks[0].page_content[:150].replace("\n"," "))

# ============================
# 4) 임베딩 + FAISS 벡터스토어
# ============================
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_store = FAISS.from_documents(chunks, embedding=embedding)


 PDF already exists: 2020_경제금융용어 700선_게시.pdf
 전체 페이지 수: 371
 청크 수: 1117
샘플 청크: iii 찾아보기 한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인  의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠  개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 .  이의 일환으로 2018년 


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
# ============================
# 5) 검색 (Top-8)
# ============================
query = "인플레이션의 정의와 원인, 금리 정책과의 관계는?"
baseline = faiss_store.similarity_search_with_relevance_scores(query, k=8)

print("\n 기본 검색 결과:")
for i, (d, s) in enumerate(baseline, 1):
    content = d.page_content[:60].replace('\n', ' ')
    print(f"{i:>2}. p.{d.metadata.get('page','?')}, score={s:.3f} | {content}...")

# ============================
# 6) LongContextReorder 적용
# ============================
try:
    from langchain_community.document_transformers import LongContextReorder
    reorder = LongContextReorder()
    reordered_docs = reorder.transform_documents([d for d, _ in baseline])
except Exception:
    from langchain.retrievers.document_compressors import LongContextReorder
    reorder = LongContextReorder()
    reordered_docs = reorder.compress_documents(documents=[d for d, _ in baseline], query=query)

# ============================
# 7) 전후 비교 출력
# ============================
id2score = {(d.metadata.get("page","?"), d.page_content[:50]): s for d, s in baseline}

print("\nLongContextReorder 적용 후 순서:")
for i, d in enumerate(reordered_docs, 1):
    key = (d.metadata.get("page","?"), d.page_content[:50])
    s = id2score.get(key, None)
    txt = d.page_content[:80].replace("\n", " ")
    score_str = f"{s:.3f}" if s is not None else "nan"
    print(f"{i:>2}. p.{d.metadata.get('page','?')}, orig_score={score_str} | {txt}...")

# ============================
# 8) 전/후 순서 비교 요약
# ============================
def label(d):
    content = d.page_content[:18].replace('\n', ' ')
    return f"p.{d.metadata.get('page','?')}:{content}"

print("\n 비교 요약")
print("기본 순서:", [label(d) for d, _ in baseline])
print("재배치  :", [label(d) for d in reordered_docs])


 기본 검색 결과:
 1. p.338, score=0.739 | 유의하는 것으로 규정하고 있다 .   연관검색어 : 중앙은행, 금융안정, 물가안정목표제...
 2. p.281, score=0.731 | 장외거래의 경우 금융위원회로부터 금융투자상품거래청산업 인가를 받은 한국거래소와  한국예탁결제원이 각각 장외파...
 3. p.154, score=0.722 | 당시 유가 상승으로 인해 생산비가 증가하면서 생산활동이 위축되고 이로 인해 경기가  둔화되었다.   연관검색...
 4. p.348, score=0.714 | 2017년 10월까지 환율관찰대상국으로 분류된 바 있다 .  연관검색어 : 스무딩오퍼레이션...
 5. p.89, score=0.709 | 를 하는 수입국은 해당 물품의 수출국에 적절한 보상을 해줄 것을 권고하고 있다 .   연관검색어 : 상계관세...
 6. p.23, score=0.707 | 기관의 영업활동이 전 세계적으로 이루어지고 있는데 반해 규제 ･감독이 금융기관의  국적 또는 지역기반을 중심...
 7. p.299, score=0.689 | 경제활동은 해당연도의 이익만을 위한 것이 아니고 미래의 지속적인 수입의 보장 등  장기적인 안목에서 이루어진...
 8. p.143, score=0.649 | 공급된 자금은 상당부분이 금융기관에 예금 등으로 다시 유입되고, 금융기관은 그 가운데  필요 지급준비금을 제...

LongContextReorder 적용 후 순서:
 1. p.281, orig_score=0.731 | 장외거래의 경우 금융위원회로부터 금융투자상품거래청산업 인가를 받은 한국거래소와  한국예탁결제원이 각각 장외파생상품거래(원화 이자율스왑거래)와 주...
 2. p.348, orig_score=0.714 | 2017년 10월까지 환율관찰대상국으로 분류된 바 있다 .  연관검색어 : 스무딩오퍼레이션...
 3. p.23, orig_score=0.707 | 기관의 영업활동이 전 세계적으로 이루어지고 있는데 반해 규제 ･감독이 금융기관의